## Render Decision Boundaries on Map.ipynb

In [2]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import sys
import os

# Add your path
sys.path.append('..')
from geometric_dl_utils import *
from plane_folding_utils import *

# Configuration
model_path = '../models/8_2.pth'
graphics_dir = '/Users/stephen/Stephencwelch Dropbox/welch_labs/backprop_3/graphics/'
map_image_path = graphics_dir + '/baarle_hertog_maps/baarle_hertog_maps-17.png'
output_dir = graphics_dir + '/modified_maps/'

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Load model
model = BaarleNet([8])
model.load_state_dict(torch.load(model_path))
model.eval()

BaarleNet(
  (model): Sequential(
    (0): Linear(in_features=2, out_features=8, bias=True)
    (1): ReLU()
    (2): Linear(in_features=8, out_features=2, bias=True)
  )
)

In [6]:
# Extract first layer weights and biases
with torch.no_grad():
    w1 = model.model[0].weight.cpu().numpy()  # Shape: [8, 2]
    b1 = model.model[0].bias.cpu().numpy()    # Shape: [8]

# Load the base map image
base_image = Image.open(map_image_path)
img_array = np.array(base_image)

def get_relu_decision_line(w0, w1, b, extent=1):
    """
    Get the decision boundary line for a ReLU neuron: w0*x + w1*y + b = 0
    Returns line endpoints within the extent bounds.
    """
    # The decision boundary is w0*x + w1*y + b = 0
    # Rearrange to y = -(w0*x + b)/w1 if w1 != 0
    # or x = -(w1*y + b)/w0 if w0 != 0
    
    points = []
    
    if abs(w1) > 1e-8:  # Line is not vertical
        # Check intersections with x = -extent and x = extent
        for x in [-extent, extent]:
            y = -(w0 * x + b) / w1
            if -extent <= y <= extent:
                points.append([x, y])
        
        # Check intersections with y = -extent and y = extent
        for y in [-extent, extent]:
            x = -(w1 * y + b) / w0 if abs(w0) > 1e-8 else 0
            if -extent <= x <= extent and len(points) < 2:
                # Avoid duplicate points
                if not any(np.allclose([x, y], p, atol=1e-6) for p in points):
                    points.append([x, y])
    
    elif abs(w0) > 1e-8:  # Line is vertical
        x = -b / w0
        if -extent <= x <= extent:
            points = [[x, -extent], [x, extent]]
    
    return np.array(points) if len(points) >= 2 else None

In [26]:
# def create_map_with_relu_lines(neuron_indices=None, save_individual=True, save_combined=True):
#     """
#     Create maps with ReLU decision boundaries overlaid.
    
#     Args:
#         neuron_indices: List of neuron indices to include. If None, uses all 8 neurons.
#         save_individual: Whether to save individual maps for each neuron
#         save_combined: Whether to save a combined map with all lines
#     """
    
#     if neuron_indices is None:
#         neuron_indices = list(range(8))
    
#     extent = 1.0  # Match your manim extent
    
#     # Get original image dimensions
#     img_height, img_width = img_array.shape[:2]
    
#     if save_individual:
#         # Create individual maps for each neuron
#         for neuron_idx in neuron_indices:
#             # Create figure with exact image dimensions (no borders)
#             fig, ax = plt.subplots(1, 1, figsize=(img_width/100, img_height/100), dpi=100)
#             ax.imshow(img_array, extent=[-extent, extent, -extent, extent], origin='lower')
            
#             # Get decision line for this neuron
#             line_points = get_relu_decision_line(w1[neuron_idx, 0], w1[neuron_idx, 1], b1[neuron_idx], extent)
            
#             if line_points is not None and len(line_points) >= 2:
#                 ax.plot(line_points[:, 0], line_points[:, 1], 
#                        color='white', linewidth=12, linestyle=(0, (2, 1)), alpha=0.9)
            
#             ax.set_xlim(-extent, extent)
#             ax.set_ylim(-extent, extent)
#             ax.axis('off')
            
#             # Remove all margins and padding
#             plt.subplots_adjust(left=0, right=1, top=1, bottom=0)
            
#             # Save individual map with exact original dimensions
#             output_path = os.path.join(output_dir, f'map_neuron_{neuron_idx}.png')
#             plt.savefig(output_path, dpi=100, bbox_inches='tight', pad_inches=0)
#             plt.close()
#             print(f'Saved: {output_path}')
    
#     if save_combined:
#         # Create combined map with all decision lines
#         fig, ax = plt.subplots(1, 1, figsize=(img_width/100, img_height/100), dpi=100)
#         ax.imshow(img_array, extent=[-extent, extent, -extent, extent], origin='lower')
        
#         for neuron_idx in neuron_indices:
#             line_points = get_relu_decision_line(w1[neuron_idx, 0], w1[neuron_idx, 1], b1[neuron_idx], extent)
            
#             if line_points is not None and len(line_points) >= 2:
#                 ax.plot(line_points[:, 0], line_points[:, 1], 
#                        color='white', linewidth=12, linestyle=(0, (2, 1)), alpha=0.9)
        
#         ax.set_xlim(-extent, extent)
#         ax.set_ylim(-extent, extent)
#         ax.axis('off')
        
#         # Remove all margins and padding
#         plt.subplots_adjust(left=0, right=1, top=1, bottom=0)
        
#         # Save combined map
#         output_path = os.path.join(output_dir, 'map_all_neurons.png')
#         plt.savefig(output_path, dpi=100, bbox_inches='tight', pad_inches=0)
#         plt.close()
#         print(f'Saved combined map: {output_path}')

In [28]:
def create_map_with_relu_lines(neuron_indices=None, save_individual=True, save_combined=True):
    """
    Create maps with ReLU decision boundaries overlaid.
    
    Args:
        neuron_indices: List of neuron indices to include. If None, uses all 8 neurons.
        save_individual: Whether to save individual maps for each neuron
        save_combined: Whether to save a combined map with all lines
    """
    
    if neuron_indices is None:
        neuron_indices = list(range(8))
    
    extent = 1.0  # Match your manim extent
    
    # Get original image dimensions
    img_height, img_width = img_array.shape[:2]
    
    if save_individual:
        # Create individual maps for each neuron
        for neuron_idx in neuron_indices:
            # Create figure with exact image dimensions (no borders)
            fig, ax = plt.subplots(1, 1, figsize=(img_width/100, img_height/100), dpi=100)
            
            # Display image with 50% opacity
            ax.imshow(img_array, extent=[-extent, extent, -extent, extent], origin='lower', alpha=0.5)
            
            # Get decision line for this neuron
            line_points = get_relu_decision_line(w1[neuron_idx, 0], w1[neuron_idx, 1], b1[neuron_idx], extent)
            
            if line_points is not None and len(line_points) >= 2:
                ax.plot(line_points[:, 0], line_points[:, 1], 
                       color='white', linewidth=12, linestyle=(0, (2, 1)), alpha=0.9)
            
            ax.set_xlim(-extent, extent)
            ax.set_ylim(-extent, extent)
            ax.axis('off')
            
            # Set transparent background
            fig.patch.set_alpha(0.0)
            ax.patch.set_alpha(0.0)
            
            # Remove all margins and padding
            plt.subplots_adjust(left=0, right=1, top=1, bottom=0)
            
            # Save individual map with transparency
            output_path = os.path.join(output_dir, f'map_neuron_{neuron_idx}.png')
            plt.savefig(output_path, dpi=100, bbox_inches='tight', pad_inches=0, transparent=True)
            plt.close()
            print(f'Saved: {output_path}')
    
    if save_combined:
        # Create combined map with all decision lines
        fig, ax = plt.subplots(1, 1, figsize=(img_width/100, img_height/100), dpi=100)
        
        # Display image with 50% opacity
        ax.imshow(img_array, extent=[-extent, extent, -extent, extent], origin='lower', alpha=0.5)
        
        for neuron_idx in neuron_indices:
            line_points = get_relu_decision_line(w1[neuron_idx, 0], w1[neuron_idx, 1], b1[neuron_idx], extent)
            
            if line_points is not None and len(line_points) >= 2:
                ax.plot(line_points[:, 0], line_points[:, 1], 
                       color='white', linewidth=12, linestyle=(0, (2, 1)), alpha=0.9)
        
        ax.set_xlim(-extent, extent)
        ax.set_ylim(-extent, extent)
        ax.axis('off')
        
        # Set transparent background
        fig.patch.set_alpha(0.0)
        ax.patch.set_alpha(0.0)
        
        # Remove all margins and padding
        plt.subplots_adjust(left=0, right=1, top=1, bottom=0)
        
        # Save combined map with transparency
        output_path = os.path.join(output_dir, 'map_all_neurons.png')
        plt.savefig(output_path, dpi=100, bbox_inches='tight', pad_inches=0, transparent=True)
        plt.close()
        print(f'Saved combined map: {output_path}')

In [29]:
print("Generating maps with ReLU decision boundaries...")
print(f"Model has {len(w1)} neurons in first layer")
print("First layer weights shape:", w1.shape)
print("First layer biases shape:", b1.shape)

# Create individual maps for each neuron AND a combined map
create_map_with_relu_lines(
    neuron_indices=None,  # Use all neurons
    save_individual=True,  # Save individual neuron maps
    save_combined=True     # Save combined map with all lines
)

print(f"Maps saved to: {output_dir}")

Generating maps with ReLU decision boundaries...
Model has 8 neurons in first layer
First layer weights shape: (8, 2)
First layer biases shape: (8,)
Saved: /Users/stephen/Stephencwelch Dropbox/welch_labs/backprop_3/graphics//modified_maps/map_neuron_0.png
Saved: /Users/stephen/Stephencwelch Dropbox/welch_labs/backprop_3/graphics//modified_maps/map_neuron_1.png
Saved: /Users/stephen/Stephencwelch Dropbox/welch_labs/backprop_3/graphics//modified_maps/map_neuron_2.png
Saved: /Users/stephen/Stephencwelch Dropbox/welch_labs/backprop_3/graphics//modified_maps/map_neuron_3.png
Saved: /Users/stephen/Stephencwelch Dropbox/welch_labs/backprop_3/graphics//modified_maps/map_neuron_4.png
Saved: /Users/stephen/Stephencwelch Dropbox/welch_labs/backprop_3/graphics//modified_maps/map_neuron_5.png
Saved: /Users/stephen/Stephencwelch Dropbox/welch_labs/backprop_3/graphics//modified_maps/map_neuron_6.png
Saved: /Users/stephen/Stephencwelch Dropbox/welch_labs/backprop_3/graphics//modified_maps/map_neuron_7

TypeError: create_map_with_relu_lines() got an unexpected keyword argument 'save_individual'

In [30]:
def create_lines_only(neuron_indices=None):
    """Create maps with just the ReLU lines on transparent background"""
    if neuron_indices is None:
        neuron_indices = list(range(8))
    
    extent = 1.0
    img_height, img_width = img_array.shape[:2]
    
    # Individual lines-only maps
    for neuron_idx in neuron_indices:
        fig, ax = plt.subplots(1, 1, figsize=(img_width/100, img_height/100), dpi=100)
        
        # No background image - just transparent
        ax.set_xlim(-extent, extent)
        ax.set_ylim(-extent, extent)
        
        # Get decision line for this neuron
        line_points = get_relu_decision_line(w1[neuron_idx, 0], w1[neuron_idx, 1], b1[neuron_idx], extent)
        
        if line_points is not None and len(line_points) >= 2:
            ax.plot(line_points[:, 0], line_points[:, 1], 
                   color='white', linewidth=12, linestyle=(0, (2, 1)), alpha=0.9)
        
        ax.axis('off')
        
        # Set fully transparent background
        fig.patch.set_alpha(0.0)
        ax.patch.set_alpha(0.0)
        
        plt.subplots_adjust(left=0, right=1, top=1, bottom=0)
        
        output_path = os.path.join(output_dir, f'line_only_neuron_{neuron_idx}.png')
        plt.savefig(output_path, dpi=100, bbox_inches='tight', pad_inches=0, transparent=True)
        plt.close()
        print(f'Saved lines-only: {output_path}')
    
    # Combined lines-only map
    fig, ax = plt.subplots(1, 1, figsize=(img_width/100, img_height/100), dpi=100)
    
    ax.set_xlim(-extent, extent)
    ax.set_ylim(-extent, extent)
    
    for neuron_idx in neuron_indices:
        line_points = get_relu_decision_line(w1[neuron_idx, 0], w1[neuron_idx, 1], b1[neuron_idx], extent)
        
        if line_points is not None and len(line_points) >= 2:
            ax.plot(line_points[:, 0], line_points[:, 1], 
                   color='white', linewidth=12, linestyle=(0, (2, 1)), alpha=0.9)
    
    ax.axis('off')
    fig.patch.set_alpha(0.0)
    ax.patch.set_alpha(0.0)
    
    plt.subplots_adjust(left=0, right=1, top=1, bottom=0)
    
    output_path = os.path.join(output_dir, 'lines_only_all_neurons.png')
    plt.savefig(output_path, dpi=100, bbox_inches='tight', pad_inches=0, transparent=True)
    plt.close()
    print(f'Saved combined lines-only: {output_path}')

print("\nCreating lines-only versions...")
create_lines_only()


Creating lines-only versions...
Saved lines-only: /Users/stephen/Stephencwelch Dropbox/welch_labs/backprop_3/graphics//modified_maps/line_only_neuron_0.png
Saved lines-only: /Users/stephen/Stephencwelch Dropbox/welch_labs/backprop_3/graphics//modified_maps/line_only_neuron_1.png
Saved lines-only: /Users/stephen/Stephencwelch Dropbox/welch_labs/backprop_3/graphics//modified_maps/line_only_neuron_2.png
Saved lines-only: /Users/stephen/Stephencwelch Dropbox/welch_labs/backprop_3/graphics//modified_maps/line_only_neuron_3.png
Saved lines-only: /Users/stephen/Stephencwelch Dropbox/welch_labs/backprop_3/graphics//modified_maps/line_only_neuron_4.png
Saved lines-only: /Users/stephen/Stephencwelch Dropbox/welch_labs/backprop_3/graphics//modified_maps/line_only_neuron_5.png
Saved lines-only: /Users/stephen/Stephencwelch Dropbox/welch_labs/backprop_3/graphics//modified_maps/line_only_neuron_6.png
Saved lines-only: /Users/stephen/Stephencwelch Dropbox/welch_labs/backprop_3/graphics//modified_map

In [ ]:




# print(f"Maps saved to: {output_dir}")

# # Optional: Create a version with just a subset of neurons for cleaner visualization
# print("\nCreating subset version with first 4 neurons...")
# create_map_with_relu_lines(
#     neuron_indices=[0, 1, 2, 3],
#     save_individual=False,
#     save_combined=True
# )

# # Rename the combined subset file
# if os.path.exists(os.path.join(output_dir, 'map_all_neurons.png')):
#     os.rename(
#         os.path.join(output_dir, 'map_all_neurons.png'),
#         os.path.join(output_dir, 'map_subset_neurons.png')
#     )
#     print(f"Saved subset version: {os.path.join(output_dir, 'map_subset_neurons.png')}")